<a href="https://colab.research.google.com/github/ZDDATApol4/prediction-political-preferences/blob/main/Final_GUI_projekt_ko%C5%84cowy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#@title <----- Najpierw uruchom to

%%capture
!git clone https://github.com/ZDDATApol4/prediction-political-preferences

!pip install spacy==3
!python -m spacy download pl_core_news_sm

import tweepy
import pandas as pd
import time
import keras
import spacy
import numpy as np
import string
from string import digits
import re
from keras_preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import load_model
from plotly.offline import iplot

# Twitter_Credentials

consumer_key = "5MzB6K9I3B0hVF0BOs2UjybSz"
consumer_secret = "hiPjP10G7bTuomHflC1vCWzwcdsAB444WkE0MkSZwGGfALoxBS"
access_token = "1437020925040513029-s6Dqtv9lPyufnlKtzm9tF3BQEQ8FCA"
access_token_secret = "D9SUsW12b1ya9LuWPNNmGmU0NhKIJzoeFfHsIy4xgdJ7x"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

tweets = []

def username_tweets_to_csv(username,count):
    try:      
        # Creation of query method using parameters
        tweets = tweepy.Cursor(api.user_timeline,id=username).items(count)

        # Pulling information from tweets iterable object
        tweets_list = [[tweet.user.screen_name, tweet.created_at, tweet.id, tweet.text] for tweet in tweets]

        # Creation of dataframe from tweets list
        tweets_df = pd.DataFrame(tweets_list,columns=['Author', 'Datetime', 'Tweet Id', 'Text'])
                      
        return tweets_df

    except BaseException as e:
          print('failed on_status,',str(e))
          time.sleep(3)

def lemmatize_text(text_corpus):
  lemmatized_text = []
  for tweet in tqdm(text_corpus):
    doc = nlp(tweet)
    doc_lemmatized = []
    for x in doc:
      if x.is_punct == False and x.is_stop == False:
        doc_lemmatized.append(x.lemma_)
    lemmatized_text.append(doc_lemmatized)
  return lemmatized_text

def tokenize(num_words, lemmatized_text, maxlen):
  tokenizer = Tokenizer(num_words=num_words)
  tokenizer.fit_on_texts(lemmatized_text)
  sequences = tokenizer.texts_to_sequences(lemmatized_text)
  padded = pad_sequences(sequences, maxlen=maxlen)
  print(padded.shape)
  return padded

nlp = spacy.load('pl_core_news_sm')

In [ ]:
#@title Wpisz Twitter ID i naciśnij PLAY

ID = "" #@param ["", ""] {allow-input: true}

count = 1000
data = username_tweets_to_csv(ID, count)

#Obróbka danych

data = data[data['Datetime'] >= pd.to_datetime('2021-08-01')]
test_text_corpus = data['Text']

# Remove @replies from source and target text
regex = r'(@\w+\s)+'
test_text_corpus = test_text_corpus.apply(lambda x: re.sub(regex, '', x))

# Remove URLS from source and target text
regex = r'https?:\/\/.*[\r\n]*'
test_text_corpus = test_text_corpus.apply(lambda x: re.sub(regex, '', x, flags=re.MULTILINE))

# convert source and target text to Lowercase 
test_text_corpus = test_text_corpus.apply(lambda x: x.lower())

# Remove quotes from source and target text
test_text_corpus = test_text_corpus.apply(lambda x: re.sub("'", '', x))

# create a set of all special characters
special_characters= set(string.punctuation)
# Remove all the special characters
test_text_corpus = test_text_corpus.apply(lambda x: ''.join(char1 for char1 in x if char1 not in special_characters))

# Remove digits from source and target sentences
num_digits= str.maketrans('','', digits)
test_text_corpus = test_text_corpus.apply(lambda x: x.translate(num_digits))

#Lematyzacja
test_lemmatized = lemmatize_text(test_text_corpus)

#Padding
test_padded = tokenize(num_words=50000, lemmatized_text=test_lemmatized, maxlen=20)

#Wczytanie_modelu
loaded_model = load_model('prediction-political-preferences/test_model_for_dash.h5')

predicted = loaded_model.predict(test_padded)

df = pd.DataFrame(predicted, columns=['KO', 'Konfederacja', 'Lewica', 'PSL', 'PiS'])

#Wykres
pd.plotting.register_matplotlib_converters()

pie = df.mean()
labels = ['KO', 'Konfederacja', 'Lewica', 'PSL', 'PiS']
fig = {
    'data': [
      {
       'values': pie,
       'labels': labels,
       'domain': {'x': [0, .5]},
       'name': '',
       'hoverinfo': 'label+percent+name',
       'hole': .3,
       'type': 'pie'
       }, ],
     'layout': {
         'legend_x': 0.5,
         'legend_y': 0.5,
         'title': f'Preferencje polityczne użytkownika "{ID}" na podstawie tweetów',
         'title_x': 0.06,
         'title_y': 0.9,
     }  
}
iplot(fig)